## Install Dependencies and Download dataset

In [15]:
#!python3 -m pip install transformers
#!python3 -m pip install sentencepiece
#!git clone https://github.com/YifanZhou1999/NLP_Project_datasets_fall22.git
!mkdir data
!mkdir models

## Imports

In [16]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig


def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w


## Read in the Data

In [17]:
#data_file="/content/NLP_Project_datasets_fall22/dataset2.csv"
#data_file="./dataset2.csv"
data_file="balanced_EI_dataset.csv"
dataf = pd.read_csv(data_file).dropna()
print('File has {} rows and {} columns'.format(dataf.shape[0],dataf.shape[1]))
dataf.head()

File has 185310 rows and 2 columns


,type,post
0,INFJ,What has been the most life-changing experienc...
1,INFJ,May the PerC Experience immerse you.
2,INFJ,Hello ENFJ7. Sorry to hear of your distress. I...
3,INFJ,Welcome and stuff.
4,INFJ,"Prozac, wellbrutin, at least thirty minutes of..."


In [18]:
def labelconv(x):
    return 1 if x[0] == 'E' else 0

dataf['EI'] = dataf['type'].map(lambda x:labelconv(x))
print(len(dataf['post']), len(dataf['EI']))
dataf.head()

185310 185310


,type,post,EI
0,INFJ,What has been the most life-changing experienc...,0
1,INFJ,May the PerC Experience immerse you.,0
2,INFJ,Hello ENFJ7. Sorry to hear of your distress. I...,0
3,INFJ,Welcome and stuff.,0
4,INFJ,"Prozac, wellbrutin, at least thirty minutes of...",0


## Setting up pretrained BERT Model and Tokenizer

- Use distilbert-base-uncased
- numclass = 2

*   列表项
*   列表项



In [19]:
from transformers import *
from transformers import DistilBertTokenizer, TFDistilBertModel, BertConfig

num_classes = 2

bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_model_EI = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)

loading file vocab.txt from cache at /Users/zhouyifan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/zhouyifan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /Users/zhouyifan/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads"

## Tokenize

~ 10 minues to run

In [20]:
input_ids=[]
attention_masks=[]

sentences = dataf['post']
print("Progress:")
for id, sent in enumerate(sentences):
    print(id, end="\r")
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(dataf['EI'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Progress:


/Users/zhouyifan/.pyenv/versions/3.9.13/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
len(input_ids),len(attention_masks),len(labels)

(185310, 185310, 185310)

## Saving and Loading Data into the pickle files

In [22]:
print('Preparing the pickle file.....')

pickle_inp_path='./data/bert_inp.pkl'
pickle_mask_path='./data/bert_mask.pkl'
pickle_label_path='./data/bert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)


Preparing the pickle file.....
Pickle files saved as  ./data/bert_inp.pkl ./data/bert_mask.pkl ./data/bert_label.pkl


In [23]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (185310, 64) Attention mask shape (185310, 64) Input label shape (185310,)


## Split Data

In [24]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (148248, 64) Val input shape (37062, 64)
Train label shape (148248,) Val label shape (37062,)
Train attention mask shape (148248, 64) Val attention mask shape (37062, 64)


## Training Setup
- Loss
- Optimizer
- Other metrics

In [25]:
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model',bert_model_EI.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model_EI.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________

Bert Model None


## Training the Model

In [13]:
# Check Device
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("\n\n")
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16187636843698942967
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 18244839580041451167
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]



Found GPU at: /device:GPU:0


2022-11-30 14:12:07.723580: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-30 14:12:07.723624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-30 14:12:07.725059: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-30 14:12:07.725087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [26]:
history=bert_model_EI.fit([train_inp,train_mask],train_label,batch_size=32,epochs=50,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/50


2022-11-30 16:57:12.151307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4633/4633 [==============================] - ETA: 0s - loss: 0.6603 - accuracy: 0.5885

2022-11-30 17:10:54.753757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4633/4633 [==============================] - 914s 195ms/step - loss: 0.6603 - accuracy: 0.5885 - val_loss: 0.6472 - val_accuracy: 0.6090
Epoch 2/50
4633/4633 [==============================] - 888s 192ms/step - loss: 0.6250 - accuracy: 0.6377 - val_loss: 0.6511 - val_accuracy: 0.6025
Epoch 3/50
4633/4633 [==============================] - 881s 190ms/step - loss: 0.5582 - accuracy: 0.7024 - val_loss: 0.6796 - val_accuracy: 0.6078
Epoch 4/50
4633/4633 [==============================] - 879s 190ms/step - loss: 0.4534 - accuracy: 0.7789 - val_loss: 0.7787 - val_accuracy: 0.5985
Epoch 5/50
4633/4633 [==============================] - 879s 190ms/step - loss: 0.3483 - accuracy: 0.8397 - val_loss: 0.8950 - val_accuracy: 0.5985
Epoch 6/50
4633/4633 [==============================] - 880s 190ms/step - loss: 0.2641 - accuracy: 0.8840 - val_loss: 1.0708 - val_accuracy: 0.5928
Epoch 7/50
4633/4633 [==============================] - 884s 191ms/step - loss: 0.2036 - accuracy: 0.9137 - val_loss: 1.224

KeyboardInterrupt: 

## Evaluation